<a href="https://colab.research.google.com/github/poudyaldiksha/Data-Science-project/blob/main/Lesson_26_b2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 26: Cartograms using Folium Module

## Recap

In [ ]:
#Mounting the google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

m_df=pd.read_csv("/content/drive/MyDrive/datasets/meteorite-landings.csv")
m_df.head()

# Retrieve the rows having the year values greater than or equal to 860 and less than or equal to 2016 at the same time.
correct_years_df = m_df[(m_df["year"]>=860) & (m_df["year"]<= 2016)]
correct_years_df

#Retrieve all the rows having the 'reclong' values greater than or equal to -180 degrees and less than or equal to 180 degrees.
correct_long_df = correct_years_df[(correct_years_df["reclong"] >=-180) & (correct_years_df["reclong"]<=180)]
correct_long_df

#Remove the rows containing the 0 'reclat' and 0 'reclong' values from the 'correct_long_df'.
correct_lat_long_df = correct_long_df[~((correct_long_df['reclat'] == 0 ) & (correct_long_df['reclong'] == 0))]

### Activity 1: Checking The Missing Values

So far we have removed all the unwanted values or rows. Now it's time to check whether we have any missing values or not. The missing values are generally reported as `NaN` values. We can find the rows or columns containing the `NaN` values using either the `isnull()` or the `isna()` function. They both return `True` for the `NaN` (or null or missing) values.

Let's first check whether any column has a missing value or not.

In [ ]:
#Check whether the 'correct_lat_long_df' DataFrame has missing values or not using the 'isnull()' function.
correct_lat_long_df.isnull().sum()

,0
name,0
id,0
nametype,0
recclass,0
mass,107
fall,0
year,0
reclat,0
reclong,0
GeoLocation,0


In [ ]:
correct_lat_long_df.isna().sum()

In [ ]:
m_df.shape

(45716, 10)

In [ ]:
correct_lat_long_df.shape

(32036, 10)

In [ ]:
#Retrieve all the rows containing the missing 'mass' values in the 'correct_lat_long_df' DataFrame.

correct_lat_long_df[correct_lat_long_df['mass'].isnull() == True]

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation
12,Aire-sur-la-Lys,425,Valid,Unknown,NaN,Fell,1769.0,50.66667,2.33333,"(50.666670, 2.333330)"
38,Angers,2301,Valid,L6,NaN,Fell,1822.0,47.46667,-0.55000,"(47.466670, -0.550000)"
76,Barcelona (stone),4944,Valid,OC,NaN,Fell,1704.0,41.36667,2.16667,"(41.366670, 2.166670)"
93,Belville,5009,Valid,OC,NaN,Fell,1937.0,-32.33333,-64.86667,"(-32.333330, -64.866670)"
172,Castel Berardenga,5292,Valid,Stone-uncl,NaN,Fell,1791.0,43.35000,11.50000,"(43.350000, 11.500000)"
...,...,...,...,...,...,...,...,...,...,...
31097,Palermo,18076,Valid,Unknown,NaN,Found,1966.0,-34.55000,-58.43333,"(-34.550000, -58.433330)"
36812,San Luis,23129,Valid,H,NaN,Found,1964.0,-33.33333,-66.38333,"(-33.333330, -66.383330)"
38278,Weiyuan,24233,Valid,Mesosiderite,NaN,Found,1978.0,35.26667,104.31667,"(35.266670, 104.316670)"
41460,Yamato 792768,28117,Valid,CM2,NaN,Found,1979.0,-71.50000,35.66667,"(-71.500000, 35.666670)"




We are not going to remove these rows. Instead, we will replace all the `NaN` values in the `mass` column with the median mass value because most of the `mass` values lie between the first and the third quartile values. The second quartile (or median) is a fair representative of the values lying in the inter-quartile range, i.e., $(Q3 - Q1)$.

More importantly, the quartile values remain unaffected by the unusually very high or very low values. In other words, the outliers do not affect the quartile values because they are computed by arranging all the values in the increasing order.

Now, let's get the descriptive statistics for the `mass` column in the `correct_lat_long_df` DataFrame.

In [ ]:
correct_lat_long_df['mass'].describe()

,mass
count,3.192900e+04
mean,1.854289e+04
std,6.868495e+05
min,0.000000e+00
25%,6.500000e+00
50%,2.960000e+01
75%,2.020000e+02
max,6.000000e+07


In [ ]:
import pandas as pd
a = pd.Series(data= (1,2,3,4), index=("a","b","c","d"))
a

,0
a,1
b,2
c,3
d,4


In [ ]:
b = a.index
b

Index(['a', 'b', 'c', 'd'], dtype='object')

In [ ]:
correct_lat_long_df.isnull().sum()

,0
name,0
id,0
nametype,0
recclass,0
mass,107
fall,0
year,0
reclat,0
reclong,0
GeoLocation,0


In [ ]:
correct_lat_long_df[correct_lat_long_df["mass"].isnull()== True].index
#df[condition]   , this syntax  is to filter the dataframe

Index([   12,    38,    76,    93,   172,   204,   262,   308,   312,   320,
       ...
       31055, 31056, 31057, 31058, 31059, 31097, 36812, 38278, 41460, 45698],
      dtype='int64', length=107)

In [ ]:
 #Create a list of the indices of above rows.
row_indices = correct_lat_long_df[correct_lat_long_df['mass'].isnull() == True].index
row_indices

Index([   12,    38,    76,    93,   172,   204,   262,   308,   312,   320,
       ...
       31055, 31056, 31057, 31058, 31059, 31097, 36812, 38278, 41460, 45698],
      dtype='int64', length=107)

So, here we have an array containing the indices of all the rows having the `NaN` mass values. There are 107 such rows.

### Activity 2: The `loc[]` Function

Now, let's replace the missing `mass` values with the median of `mass` values. For this, we need to use the `loc[]` function from Pandas library. It is an extension to the `iloc[]` function. The `iloc[]` function takes only the row and column indices as inputs. But in addition to the row and column indices, the `loc[]` function also accepts the array of row indices and column indices as inputs. It also accepts the specific column names as inputs.

Let's create a Pandas series containing all the rows having the `NaN` mass values using the `loc[]` function. We will pass `row_indices` array and `mass` column as inputs to the `loc[]` function. Let's store the series obtained in the `missing_mass_values` variable.

In [ ]:
correct_lat_long_df.iloc[[1,2,3],[1,2,3]]

,id,nametype,recclass
1,2,Valid,H6
2,6,Valid,EH4
3,10,Valid,Acapulcoite


In [ ]:
correct_lat_long_df.loc[[55,88,99],["name"]] # loc, iloc functions are used for filtering the data frame

,name
55,Atoka
88,Baxter
99,Bensour


In [ ]:
correct_lat_long_df.loc[row_indices,"mass"]

In [ ]:
row_indices

Index([   12,    38,    76,    93,   172,   204,   262,   308,   312,   320,
       ...
       31055, 31056, 31057, 31058, 31059, 31097, 36812, 38278, 41460, 45698],
      dtype='int64', length=107)

In [ ]:
# Retrieve the missing 'mass' values from 'correct_lat_long_df' DataFrame using the 'loc[]' function.
missing_mass_values = correct_lat_long_df.loc[row_indices, 'mass']
missing_mass_values

,mass
12,NaN
38,NaN
76,NaN
93,NaN
172,NaN
...,...
31097,NaN
36812,NaN
38278,NaN
41460,NaN


Here, we have a Pandas series which contains all the `NaN` mass values. The length of this series is `107`.

---

### Activity 3: Replacing The Missing `mass` Values

Now, let's replace the missing `mass` values in the 107 rows with the median of the `mass` values. Again, we will use the `loc[]` function for this exercise.

Using the `loc[]` function with `row_indices` and `mass` as inputs, we will get all the `NaN` values in the `mass` column. Then, using the `median()` function, we will get the median value which we are going to assign to the rows having the missing `mass` values in the `correct_lat_long_df`.


In [ ]:
y= pd.Series([1,2,3,4,5])
a =y.median()
a


3.0

In [ ]:
y[1]=a
y

,0
0,1
1,3
2,3
3,4
4,5


In [ ]:
print(a)

In [ ]:
# Replace the missing values in the 'mass' column in the 'correct_lat_long_df' DataFrame with median of mass.
median_mass = correct_lat_long_df['mass'].median()
median_mass

29.6

In [ ]:
correct_lat_long_df.loc[row_indices, 'mass'] = median_mass

Now, let's check whether all the missing mass values have been replaced by the median of the mass values or not. We can do this with the help of the `loc[]` function.

In [ ]:
#Check whether all the missing mass values have been replaced by the median of the mass values or not.
correct_lat_long_df.loc[row_indices, "mass"]

,mass
12,29.6
38,29.6
76,29.6
93,29.6
172,29.6
...,...
31097,29.6
36812,29.6
38278,29.6
41460,29.6


Each item in the `mass` column in the above DataFrame of 107 rows is `29.6` grams.

Let's confirm it by getting the descriptive statistics for the `mass` column in the above DataFrame using the `describe()` function.

In [ ]:
# Create a descriptive statistics summary of the 'mass' column in the above DataFrame containing 107 rows.
correct_lat_long_df.loc[row_indices, 'mass'].describe()

,mass
count,1.070000e+02
mean,2.960000e+01
std,5.711092e-14
min,2.960000e+01
25%,2.960000e+01
50%,2.960000e+01
75%,2.960000e+01
max,2.960000e+01


In [ ]:
#  Create a descriptive statistics summary of the 'mass' column in the 'correct_lat_long_df' DataFrame.
correct_lat_long_df['mass'].describe()

,mass
count,3.203600e+04
mean,1.848105e+04
std,6.857023e+05
min,0.000000e+00
25%,6.500000e+00
50%,2.960000e+01
75%,2.006500e+02
max,6.000000e+07


As you can see there is no major change in the `std, min, 25%, 50%, 75%` and `max` values. They all are the same which is how it should be. **Treatment of the missing values should not affect the original dataset drastically.**



---

##**Cartograms**

We will create a cartogram using the Folium package. There are other Python packages to create cartograms but Folium is the easiest one to get started.

A cartogram is simply a world map marked with the country-wise or state-wise trend in data. We will create a cartogram for the landing sites of meteorites. It will have markers for the places from where a meteorite was found to pin-point the meteorite landing site.

### Activity 4: Cartogram Using The `folium` Module

Now, let's a simple map of Mumbai to get started with cartograms. The location coordinates of the Mumbai city is `19.0760` and `72.8777`. To get the coordinates of a city, just google `city_name coordinates`.

To create a cartogram (or a map) you first need to import the `folium` module.

```
import folium
```

Then you need to call the `Map()` function which exists in the `folium` module. It mandatorily requires the coordinates (latitude and longitude) of a place.

```
folium.Map(location=[latitude, longitude])


In [ ]:
a = folium.Map(location=[0,0])
a

In [ ]:
#Create a map for the location having the coordinates: 19.0760 and 72.8777.
import folium
mumbai_map = folium.Map(location = [19.0760, 72.8777])
mumbai_map

In [ ]:

nepal_map = folium.Map(location = [28.3949,84.140])
nepal_map

#### Activity 4.1: The `width` & `height` Parameters

You can also set the size of the map by providing two more inputs to the `folium.Map()` function. They are the `width` and `height` values which vary between `'0%'` and `'100%'` for each parameter.

In [ ]:
# Resize the Mumbai map by setting the width='90%' and height='70%' inside the 'folium.Map()' function.
mumbai_map = folium.Map(location = [19.0760, 72.8777], width="30%", height="30%")
mumbai_map

---

#### Activity 4.2: The `zoom_start` Parameter

You can also set the initial zoom value using the `zoom_start` parameter inside the `folium.Map()` function, if you want to start the map with either a closer view or a farther view of the location in the map.

In [ ]:
#Set the 'zoom_start' value equal to 15 inside the 'folium.Map()' function to get a closer view of Mumbai right from the start.
# The number '15' is chosen randomly. You can set any 'zoom_start' value you wish to.
mumbai_map = folium.Map(location = [19.0760, 72.8777], width="90%", height="70%",zoom_start = 15)
mumbai_map

---

#### Activity 4.3: Adding A Marker

Now, let's add a marker to the Mumbai map. If you search for the **Gateway of India** on Google maps, you will get a red coloured marker as shown in the image below.


We can also add a marker like these in a cartogram. Let's mark the Gateway of India (coordinates `18.9220, 72.8347`) on the Mumbai map using a marker.

To add a marker, you need to use the `folium.Marker()` function. It requires location coordinates of the place to be marked as an input.

```
folium.Marker(location=[latitude, longitude])
```

You can also add another parameter called `popup` inside the `folium.Marker()` function to popup some information on the marker when a user clicks on it. Note that it is not mandatory to provide the value to the `popup` parameter.

```
folium.Marker(location=[latitude, longitude], popup='Some information')
```

At the end of the `folium.Marker()` function, you need to apply the `add_to()` function whose input is the map in which you want to add the marker.

```
folium.Marker(location=[latitude, longitude], popup='Some information').add_to(map_name)
```

Let's display `'Gateway of India` on the marker when a user clicks on it.


In [ ]:
my_nepalmap = folium.Map(location=[28.3949,84.140],zoom_start=10)
folium.Marker(location=[28.2096, 83.9856],popup = "phewa lake",icon= folium.Icon(color="green",icon='cloud')).add_to(my_nepalmap)
my_nepalmap

In [ ]:
# Create a map centered at specific coordinates (latitude, longitude)
my_map = folium.Map(location=[19.0760, 72.8777],zoom_start=3)  # Mumbai's coordinates

# Add a marker to the map
folium.Marker(
    location=[18.9220, 72.8347],  # Coordinates of Mumbai gate way
    popup='Gate way of India',
    icon=folium.Icon(color='red', icon='cloud')  # Popup message when you click the marker
    ).add_to(my_map)

# Display the map
my_map

In [ ]:
# Mark the Gateway of India (coordinates 18.9220, 72.8347) on the Mumbai map.
mumbai_map = folium.Map(location= [18.9220, 72.8347])
folium.Marker(location=[18.9220, 72.8347],popup="Gateway of India").add_to(mumbai_map)
folium.Marker(location=[19.0896, 72.8656], popup="Mumbai International Airport", icon = folium.Icon(color= "red")).add_to(mumbai_map)
mumbai_map

In [ ]:
# Create a map centered at New York City
nyc_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Add markers for some points of interest
folium.Marker(location=[40.7580, -73.9855], popup='Times Square', icon=folium.Icon(color='red')).add_to(nyc_map)
folium.Marker(location=[40.7127, -74.0134], popup='Statue of Liberty', icon=folium.Icon(color='green')).add_to(nyc_map)
folium.Marker(location=[40.730610, -73.935242], popup='Central Park', icon=folium.Icon(color='blue')).add_to(nyc_map)

# Display the map
nyc_map

**Customizing the Icon:**

You can customize the icon by changing its color and type.

In [ ]:


# Create a map centered at specific coordinates (latitude, longitude)
my_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)  # New York City's coordinates

# Add markers with different icons and colors
folium.Marker(
    location=[40.7580, -73.9855],  # Times Square
    popup='Times Square',
    icon=folium.Icon(color='red', icon='star')  # Star icon in red color
).add_to(my_map)

folium.Marker(
    location=[40.7127, -74.0134],  # Statue of Liberty
    popup='Statue of Liberty',
    icon=folium.Icon(color='green', icon='info-sign')  # Info-sign icon in green color
).add_to(my_map)

folium.Marker(
    location=[40.730610, -73.935242],  # Central Park
    popup='Central Park',
    icon=folium.Icon(color='blue', icon='cloud')  # Cloud icon in blue color
).add_to(my_map)

# Display the map
my_map

**Other Icon Types:**

Some common icon typesthat you can use include:
`
'home'
'cloud'
'star'
'heart'
'info-sign'
'warning'
'flag'`

---

#### Activity 4.4: Circular Markers

Apart from the regular marker, you can also add the circular makers to a folium map. There are two functions to add circular markers. They are as follows:

1. `folium.Circle()` - Radius of the circular marker is reflected in meters. It tells the land area occupied on the map.

2. `folium.CircleMarker()` - Radius of the circular marker is reflected in pixels. It tells the number of pixels occupied on the image of the map.

You will notice the difference in both the markers when you zoon-in and zoom-out of the map. Both the above functions take the following parameters as inputs:

1. `location` - It defines the position on the map where a circular marker needs to be placed.

2. `radius` - It defines the radius of the circular marker

3. `popup` - It defines the information to be displayed which clicked on a marker.

4. `color` - It defines the colour of the circular marker.

5. `fill` - It defines whether the circular marker should hollow or filled.

6. `fill_color` - It defines the colour of the filled circular marker.

Let's create the circular markers for the Mumbai international airport and the Dharavi slum in Mumbai. Their coordinates are `19.0896, 72.8656` and `19.0380, 72.8538` respectively. Let's colour them with blue and red colours respectively.

**Note:**

1. `#0000ff` is the hexadecimal colour code for the blue colour.

2. `#ff0000` is the hexadecimal colour code for the red colour.




In [ ]:
#Add the circular markers for the Mumbai international airport (19.0896, 72.8656) & the Mumbai Dharavi slum (19.0380, 72.8538)
mumbai_map = folium.Map(location=[19.0760,72.8777])
# Marker for the Gateway of India
folium.Marker(location=[18.9220, 72.8347],popup="Gateway of India").add_to(mumbai_map)
# Marker for the Mumbai international airport.
folium.Circle(location=[19.0896, 72.8656],
              radius= 1000,# radius is in meters
              popup="Mumbai airport",
              color="red",
              fill=False).add_to(mumbai_map)
# Marker for the Dharavi slum in Mumbai.
folium.CircleMarker(location=[19.0380, 72.8538],
                    radius= 50,# pixels
                    popup="Dharavi Slum",
                    color="blue",
                    fill= True,
                    fill_color="green").add_to(mumbai_map) # its in pixels
mumbai_map

As you zoom-in and out of the map, the radius of the circular marker created using the `folium.Circle()` function increases and decreases whereas the radius of the circular marker created using the `folium.CircleMarker()` remains the same because the latter circular marker occupies the fixed number of pixels on the map.



### Activity 5: Series datatype conversion

We need to generate cartograms to illustrate the locations of meteorite landings. Our goal is to create a world map with markers indicating where meteorites have been discovered. However, given that there are over 30,000 meteorite records, plotting all of them on a single cartogram would result in an overly cluttered map, making it difficult to interpret any meaningful information. Therefore, we'll divide the data into five-year intervals and create separate cartograms for each period. For instance, we'll make individual cartograms for the periods 2001-2005, 2006-2010, 2011-2015, and so on.




In [ ]:
correct_lat_long_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32036 entries, 0 to 45715
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         32036 non-null  object 
 1   id           32036 non-null  int64  
 2   nametype     32036 non-null  object 
 3   recclass     32036 non-null  object 
 4   mass         32036 non-null  float64
 5   fall         32036 non-null  object 
 6   year         32036 non-null  float64
 7   reclat       32036 non-null  float64
 8   reclong      32036 non-null  float64
 9   GeoLocation  32036 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 3.7+ MB


In [ ]:
correct_lat_long_df.head()

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775000, 6.083330)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.183330, 10.233330)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.216670, -113.000000)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.883330, -99.900000)"
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.166670, -64.950000)"


In [ ]:
# Convert the 'year' values into integer type values.
correct_lat_long_df["year"] = correct_lat_long_df["year"].astype("int")
correct_lat_long_df["year"]

<ipython-input-63-c3ddec828ac0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correct_lat_long_df["year"] = correct_lat_long_df["year"].astype("int")


,year
0,1880
1,1951
2,1952
3,1976
4,1902
...,...
45711,1990
45712,1999
45713,1939
45714,2003


In [ ]:
# Confirm whether the 'year' values are integer or not using the 'dtype' keyword.
correct_lat_long_df["year"].dtype

dtype('int64')

In [ ]:
correct_lat_long_df["nametype"].value_counts()

,count
nametype,
Valid,31967
Relict,69


---

### Activity 6: The Good & Withered Meteorites Separation

Our next task is to make a cartogram for the landing locations of the meteorites found on Earth. They were found in two conditions: the good and the withered condition. The condition of the meteorites is described in the `nametype` column. If the `nametype` value is `Valid`, then it means the condition of the meteorite found is good whereas if the `nametype` value is `Relict`, then it means the condition of the meteorite found is withered.

Before creating a cartogram, let's create two DataFrames.

- The first DataFrame for the meteorites which were actually found and are in good condition (`nametype` value is `Valid`).

- The second DataFrame for the meteorites which were actually found and are in a withered condition (`nametype` value is `Relict`).

In [ ]:
# Find out how many meteorites are in good condition ('nametype' == 'Valid') and how many are withered ('nametype' == 'Relict').
correct_lat_long_df["nametype"].value_counts()

,count
nametype,
Valid,31967
Relict,69


So, there are only 69 meteorites which have withered due to the prolonged exposure to probably the extreme weather conditions. The other meteorites are in good condition. Since the withered meteorites are fewer in numbers, let's create a DataFrame for the same and create its cartogram because it would be easier for us to spot these meteorites on a map.

In [ ]:
#Create a DataFrame called 'found_relict_df' and store data only for those meteorites which were actually found.
found_relict_df= correct_lat_long_df[(correct_lat_long_df["nametype"]=="Relict") & (correct_lat_long_df["fall"]=="Found")]
found_relict_df

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation
5182,Brunflo,5157,Relict,Relict H,29.6,Found,1980,63.11667,14.28333,"(63.116670, 14.283330)"
6810,David Glacier 92308,6614,Relict,Chondrite-fusion crust,1.7,Found,1992,-75.31667,162.00000,"(-75.316670, 162.000000)"
12627,Gove,52859,Relict,Relict iron,0.0,Found,1979,-12.26333,136.83833,"(-12.263330, 136.838330)"
15944,Gullhögen 001,44889,Relict,Relict OC,29.6,Found,2000,58.38333,13.80000,"(58.383330, 13.800000)"
20674,Lewis Cliff 87241,13702,Relict,Chondrite-fusion crust,0.5,Found,1987,-84.34563,161.31058,"(-84.345630, 161.310580)"
...,...,...,...,...,...,...,...,...,...,...
31072,Österplana 060,56159,Relict,Relict OC,0.0,Found,2009,58.58333,13.43333,"(58.583330, 13.433330)"
31073,Österplana 061,56160,Relict,Relict OC,0.0,Found,2009,58.58333,13.43333,"(58.583330, 13.433330)"
31074,Österplana 062,56161,Relict,Relict OC,0.0,Found,2010,58.58333,13.43333,"(58.583330, 13.433330)"
31075,Österplana 063,56162,Relict,Relict OC,0.0,Found,2010,58.58333,13.43333,"(58.583330, 13.433330)"


Now, create a DataFrame for the meteorites which were actually found and are in good condition (`nametype` value is `Valid`).

In [ ]:
found_relict_df.shape

(69, 10)

In [ ]:
# Create a DataFrame called 'found_valid_df' and store data only for those meteorites which were actually found.
found_valid_df=correct_lat_long_df[(correct_lat_long_df["fall"] == "Found") & (correct_lat_long_df["nametype"] == "Valid")]
found_valid_df

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation
1108,Abajo,4,Valid,H5,331.00,Found,1982,26.80000,-105.41667,"(26.800000, -105.416670)"
1109,Abar al' Uj 001,51399,Valid,H3.8,194.34,Found,2008,22.72192,48.95937,"(22.721920, 48.959370)"
1110,Abbott,5,Valid,H3-6,21100.00,Found,1951,36.30000,-104.28333,"(36.300000, -104.283330)"
1111,Abernathy,7,Valid,L6,2914.00,Found,1941,33.85000,-101.80000,"(33.850000, -101.800000)"
1112,Abo,8,Valid,H,1.20,Found,1840,60.43333,22.30000,"(60.433330, 22.300000)"
...,...,...,...,...,...,...,...,...,...,...
45711,Zillah 002,31356,Valid,Eucrite,172.00,Found,1990,29.03700,17.01850,"(29.037000, 17.018500)"
45712,Zinder,30409,Valid,"Pallasite, ungrouped",46.00,Found,1999,13.78333,8.96667,"(13.783330, 8.966670)"
45713,Zlin,30410,Valid,H4,3.30,Found,1939,49.25000,17.66667,"(49.250000, 17.666670)"
45714,Zubkovsky,31357,Valid,L6,2167.00,Found,2003,49.78917,41.50460,"(49.789170, 41.504600)"


So there are 30,871 meteorites which were found in good condition.

---

In [ ]:
found_relict_df

,name,id,nametype,recclass,mass,fall,year,reclat,reclong,GeoLocation
5182,Brunflo,5157,Relict,Relict H,29.6,Found,1980,63.11667,14.28333,"(63.116670, 14.283330)"
6810,David Glacier 92308,6614,Relict,Chondrite-fusion crust,1.7,Found,1992,-75.31667,162.00000,"(-75.316670, 162.000000)"
12627,Gove,52859,Relict,Relict iron,0.0,Found,1979,-12.26333,136.83833,"(-12.263330, 136.838330)"
15944,Gullhögen 001,44889,Relict,Relict OC,29.6,Found,2000,58.38333,13.80000,"(58.383330, 13.800000)"
20674,Lewis Cliff 87241,13702,Relict,Chondrite-fusion crust,0.5,Found,1987,-84.34563,161.31058,"(-84.345630, 161.310580)"
...,...,...,...,...,...,...,...,...,...,...
31072,Österplana 060,56159,Relict,Relict OC,0.0,Found,2009,58.58333,13.43333,"(58.583330, 13.433330)"
31073,Österplana 061,56160,Relict,Relict OC,0.0,Found,2009,58.58333,13.43333,"(58.583330, 13.433330)"
31074,Österplana 062,56161,Relict,Relict OC,0.0,Found,2010,58.58333,13.43333,"(58.583330, 13.433330)"
31075,Österplana 063,56162,Relict,Relict OC,0.0,Found,2010,58.58333,13.43333,"(58.583330, 13.433330)"


### Activity 7: Cartogram For Withered Meteorites

Let's create a cartogram for all the withered meteorites that were found. We need to add a marker for every single meteorite listed in the `found_relict_df`. Hence, we need to use the `for` loop to iterate through each entry in the `found_reliect_df` DataFrame. Then we also need to use the `loc[]` function to get the latitude and longitude values for each meteorite. Since we need to display the location of the meteorite site in each marker through the popup window, we again will have to use the `loc[]` function.

**Note:** If you are creating a world map, then provide the coordinates of the equator (`0, 0`) as input in the `Map()` function to create a symmetric map.

In [ ]:
for i in found_relict_df.index:
    print(i)

In [ ]:
found_relict_df.index

In [ ]:
found_relict_df.loc[31072,["name"]]

,31072
name,Österplana 060


In [ ]:
found_relict_df.loc[:,["name"]]

In [ ]:
# Create a cartogram for the landing sites of the meteorites found in the withered condition.

map1 = folium.Map(location=[0,0],width="90%",height="80%",zoom_start = 1.5)
for i in found_relict_df.index:
    folium.Marker(location=[found_relict_df.loc[i,"reclat"],found_relict_df.loc[i,"reclong"]],
                  popup= found_relict_df.loc[i,"name"]).add_to(map1)
map1

---

### Activity 8: Cartograms For Good Condition Meteorites

Now, let's make a cartogram for the meteorites which are found in good condition (`found_valid_df`). As we saw earlier, 30,871 meteorites were found in good condition. Hence, creating markers for each of these meteorites is impractical. So, we can slice the `found_valid_df` DataFrame based on year.

Let's estimate how many meteorites were found in good condition after the year 2010.

In [ ]:
found_valid_df.head()

In [ ]:
# Find out how many meteorites were found in good condition after the year 2010.
found_valid_df[found_valid_df["year"]>2010].index

In [ ]:
found_valid_df[found_valid_df['year'] > 2010].shape

So, 398 meteorites were found in good condition after the year 2010. Let's create a cartogram for these meteorites. We will use the same process which we used to create a cartogram for the withered meteorites.



In [ ]:
#  Create a cartogram for the meteorites found in good condition after the year 2010. Popup the location name.
map2 = folium.Map(location=[0.0, 0.0], width='90%', height='80%', zoom_start=1.5)

for i in found_valid_df[found_valid_df['year'] > 2010].index:
  folium.Marker(location=[found_valid_df.loc[i, 'reclat'], found_valid_df.loc[i, 'reclong']],
                popup=found_valid_df.loc[i, 'name']).add_to(map2)

map2

Similarly, let's create a cartogram for the meteorites found in good condition from 2008 to 2010 (both inclusive).

In [ ]:
#Find out how many meteorites found in good condition from 2008 to 2010 (both inclusive).

found_valid_df[(found_valid_df['year'] > 2007) & (found_valid_df['year'] <= 2010)].shape

So, 976 meteorites were found in good condition from 2008 to 2010 (both inclusive). Now, let's create a cartogram for these meteorites.

In [ ]:
#Create a cartogram for the meteorites found in good condition from 2008 to 2010 (both inclusive).
map3 = folium.Map(location=[0, 0], width='90%', height='85%', zoom_start=1.5)

for i in found_valid_df[(found_valid_df['year'] > 2007) & (found_valid_df['year'] <= 2010)].index:
  folium.Marker(location=[found_valid_df.loc[i, 'reclat'], found_valid_df.loc[i, 'reclong']],
              popup=found_valid_df.loc[i, 'name']).add_to(map3)

map3

Similarly, let's create a cartogram for the meteorites found in good condition from 2005 to 2007 (both inclusive).

In [ ]:
#  How many meteorites were found in good condition from 2005 to 2007 (both inclusive)?
found_valid_df[(found_valid_df['year'] > 2004) & (found_valid_df['year'] <= 2007)].shape

So, 1688 meteorites were found in good condition from 2005 to 2007 (both inclusive). Now, let's create a cartogram for these meteorites.



In [ ]:
# Create a cartogram for the meteorites found in good condition from 2005 to 2007 (both inclusive).
import folium
map4 = folium.Map(location=[0, 0], width='90%', height='85%', zoom_start=1.5)

for i in found_valid_df[(found_valid_df['year'] > 2004) & (found_valid_df['year'] <= 2007)].index:
  folium.Marker(location=[found_valid_df.loc[i, 'reclat'], found_valid_df.loc[i, 'reclong']],
              popup=found_valid_df.loc[i, 'name']).add_to(map4)

map4

The `popup` parameter displays the name of the location on a popup box when clicked on a marker. Along with the city name, we can also display the mass of a meteorite in the popup of the markers. For this, we will have to use **string concatenation**. It is a process in which two or more Python objects can be joined together into one string object.


---

### Activity 9: Multiple data on markers



we can concatenate each `name` value with the corresponding `mass` value and then display both the city names and the mass of a meteorite through the `popup` parameter. Let's display the name and the mass of the meteorites for the meteorites found in a condition from the year 2005 to 2007 (both inclusive).

In [ ]:
#Use the string concatenation method to display the name of the place and mass of the meteorite in the marker popup.
map5 = folium.Map(location=[0, 0], width='90%', height='90%', zoom_start=1.5)

for i in found_valid_df[(found_valid_df['year'] > 2004) & (found_valid_df['year'] <= 2007)].index:
  folium.Marker(location=[found_valid_df.loc[i, 'reclat'], found_valid_df.loc[i, 'reclong']],
                popup=found_valid_df.loc[i, 'name'] + '\n' + str(found_valid_df.loc[i, 'mass'] / 1000) + ' kg').add_to(map5)
                # The '\n' character adds a new line. Also, we have converted the mass into kg from grams by dividing the mass by 1000.
map5

If you click on any of the markers, you will see that it displays both the city name and the mass of a meteorite. We can also add circular markers such that the radius of a circular marker represents the mass of the meteorite fallen. So, the bigger the circle, the heavier the meteorite.

In [ ]:
#  Add circular markers such that the radius of a circular marker represents the mass of the meteorite fallen.
map6 = folium.Map(location=[0, 0], width='90%', height='90%', zoom_start=1.5)

for i in found_valid_df[(found_valid_df['year'] > 2004) & (found_valid_df['year'] <= 2007)].index:
  folium.Circle(location=[found_valid_df.loc[i, 'reclat'], found_valid_df.loc[i, 'reclong']],
                radius=found_valid_df.loc[i, 'mass'],
                color='crimson',
                fill=True,
                fill_color='crimson',
                popup=found_valid_df.loc[i, 'name'] + '\n' + str(found_valid_df.loc[i, 'mass'] / 1000) + ' kg').add_to(map6)
map6

### Activity 10: Count Plots

Now let's make a count plot for all the years after 2000 to see the variation in the number of meteorites observed over the years.

In [ ]:
# Create a DataFrame called 'm_2000_df' and store data for the meteorites discovered after 1990.
m_2000_df = correct_lat_long_df[correct_lat_long_df ["year"] >2000]
m_2000_df



To create a count plot, we will use the `countplot()` function which exists in the `seaborn` module. It takes two inputs. The first input is the axis on which the values to be plotted and the second input is the source of the data.

In [ ]:
m_2000_df.year

In [ ]:
m_2000_df["year"]

In [ ]:
plt.figure(figsize=(20,5))
sns.countplot(x="year",data=m_2000_df)
plt.show()

In [ ]:
# Create a count plot
import seaborn as sns
sns.countplot(x= m_2000_df["year"])
plt.show()

---

### Activity 11: Horizontal Bars In Count Plot

We can also change the orientation of a count plot by plotting it horizontally.

In [ ]:
# Create a horizontal count plot
plt.figure(figsize=(20,5))
sns.countplot(y="year",data=m_2000_df)
plt.show()


---

### Activity 12: The `hue` Parameter
We can differentiate the count bars according to a specific category in a count plot. For instance, we can display:

1. The number of meteorites that fell and were not discovered in a given year, and
2. The number of meteorites that fell and were discovered in the same year.
To achieve this differentiation in the count bars, we use the hue parameter.

In [ ]:
# Split the count bar for a year into the 'Fell' bar and the 'Found' bar.
plt.figure(figsize=(20,5))
sns.countplot(y="year",data=m_2000_df, hue="fall")
plt.show()


In [ ]:
#Split the count bar for year into the 'Valid' bar and the 'Relict' bar.
plt.figure(figsize=(20,5))
sns.countplot(y="year",data=m_2000_df,hue="nametype")
plt.show()


### Activity 13: Histograms

Count plots are useful for visualizing the distribution of a categorical feature across different categories. They are particularly effective when you want to compare the count of each category in a dataset.

However, when dealing with large datasets and you want to observe the distribution of a single numerical feature, it is more effective to use a histogram.

A histogram is a type of bar graph that groups data into bins or intervals, showing the frequency of data points within each bin. This allows you to see the overall distribution of the data.

**Key Points:**
- Count Plot: Ideal for visualizing and comparing the frequency of categories in a categorical variable. It displays the count of occurrences for each category.

- Histogram: Suitable for understanding the distribution of a numerical variable. It groups data points into bins and shows how many points fall into each bin.

In [ ]:
# Generating random numerical data
data = np.random.randn(1000)
data[:11]


The `numpy.random.randn` function generates samples from the standard normal distribution. This means the generated values will follow a normal distribution (bell curve) with a mean of `0` and a standard deviation of `1`.

Theoretically, the values generated by randn can range from negative infinity to positive infinity, but practically, most of the values will fall within a certain range around the mean due to the properties of the normal distribution. Specifically:

- Approximately 68% of the values will fall within one standard deviation of the mean (between -1 and 1).
- Approximately 95% of the values will fall within two standard deviations of the mean (between -2 and 2).
- Approximately 99.7% of the values will fall within three standard deviations of the mean (between -3 and 3).


In [ ]:
# Creating a histogram
plt.hist(data, bins=30, edgecolor='black')
plt.title('Histogram of Random Data')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

**Understanding Bins in Histograms**

Bins in histograms are used to group continuous data into intervals. Each bin represents a range of values, and the height of the bar in a bin represents the frequency of data points within that range.

In [ ]:
#Simple Histogram with Default Bins
# Generate random data
data = np.random.randn(1000)

# Create a histogram with default bins
plt.hist(data, edgecolor='black')
plt.title('Histogram with Default Bins')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Create a subplot to compare histograms with different bins
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
# 5 bins
axes[0].hist(data, bins=5, edgecolor='black')
axes[0].set_title('Histogram with 5 Bins')
axes[0].set_xlabel('Value')
axes[0].set_ylabel('Frequency')

# 20 bins
axes[1].hist(data, bins=20, edgecolor='black')
axes[1].set_title('Histogram with 20 Bins')
axes[1].set_xlabel('Value')
axes[1].set_ylabel('Frequency')

# 50 bins
axes[2].hist(data, bins=50, edgecolor='black')
axes[2].set_title('Histogram with 50 Bins')
axes[2].set_xlabel('Value')
axes[2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Define custom bin edges
#You can also define custom bin edges to have more control over the histogram intervals.
bin_edges = [0, 1, 2, 3]

# Create a histogram with custom bin edges
plt.hist(data, bins=bin_edges, edgecolor='black')
plt.title('Histogram with Custom Bins')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

Now, suppose we want to plot a bar graph for the count of the number of meteorites fallen in the interval of 5 years between 1970 and 2000. In this case, we will have 6 intervals. They are:

1. 1970 to 1975 (excluding 1975)

2. 1975 to 1980 (excluding 1980)

3. 1980 to 1985 (excluding 1985)

4. 1985 to 1990 (excluding 1990)

5. 1990 to 1995 (excluding 1995)

6. 1995 to 2000 (**including** 2000)



To create such bar graphs, we use a histogram. The bars in the count plot merge together to form one bar in the histogram. So, for example for the interval 1970 to 1975:

1. 1970 to 1975 (excluding 1975), the bars in a count plot for the years 1970, 1971, 1972, 1973 and 1974 will merge together to form one bar in a histogram.

To create a histogram, you can use the `hist()` function which exists in the `matplotlib.pyplot` library. The `hist()` function requires a one dimensional array/list/series as an input to create a histogram.

In [ ]:
#Create a Pandas series containing the year values between 1970 and 2000 including both of them.
m_1970_and_2000_series = correct_lat_long_df.loc[(correct_lat_long_df['year'] >= 1970) & (correct_lat_long_df['year'] < 2001), 'year']
m_1970_and_2000_series.shape

In [ ]:
#Create a histogram for the Pandas series containing the year values between 1970 and 2000 including both of them.
plt.figure(figsize=(15,6))
plt.hist(m_1970_and_2000_series,bins=6,color="green")
plt.grid()
plt.show()

**Note** The `bins` value to define the number of bars to be created in the histogram. Here, we are going to create 6 bars, hence we will pass `bins=6` as the second input to the `hist()` function.

### Activity 14: Histogram Using The `distplot()` Function

We can also create a histogram using the `distplot()` function from the `seaborn` module. The term `distplot` stands for distribution plot. It also takes a Pandas series / NumPy array / Python list as an input. The other inputs that you can provide are `bins` and `kde` values. The `kde` parameter takes a boolean value, i.e., either `True` or `False`. Let's set the `bins` parameter equal to `6` and the `kde` parameter equal to `False` in the `distplot()` function. [ We will learn about the `kde` parameter later ]

In [ ]:
#Create a histogram using the 'distplot()' function from the seaborn library.

plt.figure(figsize=(10, 5))
sns.distplot(m_1970_and_2000_series, bins=6, kde= False)
plt.grid()
plt.show()

In [ ]:

plt.figure(figsize=(10, 5))
sns.histplot(m_1970_and_2000_series, bins=6, kde= False)
plt.grid()
plt.show()

In [ ]:

plt.figure(figsize=(10, 5))
sns.displot(m_1970_and_2000_series)
plt.grid()
plt.show()

**Note**

As you can see, we have created the same histogram using the `distplot()` function. **The `distplot()` function is smarter in terms of choosing the appropriate default number of bins**. Hence, it is best to create a histogram using the `distplot()` function over the `hist()` function.